# AIFFEL 일곱번째 프로젝트 : 나랑 닮은 연예인을 찾아보자
## 1. 사진 모으기
- `Slack #AIFFEL와글와글` 변효윤 퍼실님이 올려주신 [셀럽사진 파일](https://drive.google.com/file/d/1SCNlkW3l9bzsGhzFVk8DmNdfq9D-sey2/view?usp=sharing)(+보충)로 사진을 모았다.
- 그리고 사진 폴더에 내 사진 2장을 `Jeongmr1`, `Jeongmr2`의 이름으로 추가했다.

## 2. 얼굴 영역 자르기
- `def get_gropped_face(image_file)` 통해서 잘라주기!
    - 자른 이미지는 PIL의 `image.fromarray` 통해서 PIL image로 변환한 뒤에 저장
        - 다양한 재미있는 시각화 시도에 도움
```
#    힌트
#    from PIL import Image
#    face = get_gropped_face(image_path)
#    pillow_image = Image.fromarray(face)
#    pillow_image.save(path_to_save)
```

In [1]:
import face_recognition
import os
import matplotlib.pyplot as plt

def get_cropped_face(image_file):

    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    if len(face_locations) != 0:        # 아래에서 설명
        a, b, c, d = face_locations[0]
        cropped_face = image[a:c,d:b,:]
    else: cropped_face = []

    return cropped_face

- 얼굴 영역 자르는 과정에서 `face_location`이 아예 잡히지 않는 사진이 종종 있어, 그런 경우는 `get_cropped_face` 함수의 리턴값을 빈 리스트로 두었다. 이는 후에 임베딩 벡터 처리에서 `len`로 거른다.

In [2]:
# 이미지 디렉토리 주소, 디렉토리 안의 파일 리스트
dir_path = os.getenv('HOME')+'/aiffel/face_embedding/celebrity'
file_list = os.listdir(dir_path)

## 3. 얼굴 영역 임베딩 추출하기
- 이전에 만든 함수 사용하기
- `def get_face_embedding(face)` : 얼굴영역으로부터 얼굴 임베딩 벡터를 구하는 함수
- `def get_face_embedding_dict(dir_path)` : 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수

In [3]:
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

In [4]:
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_cropped_face(img_path)
        if len(face) == 0: continue
        embedding = get_face_embedding(face)
        if len(embedding) > 0:
            # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
            # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
        
    return embedding_dict

In [5]:
embedding_dict = get_face_embedding_dict(dir_path)
embedding_dict.keys()

dict_keys(['류준열', '이동휘', '잭 블랙', '김연우', '시우민', '앤드류 응', '이제훈', '배성우', '조유리', '고창석', 'RM', '차두리', '변요한', '세훈', '이연걸', '나인뮤지스경리', '레드벨벳웬디', '신동엽', '숀 코네리', '지수', '박찬호', '이용진', '이대호', '제이콥 배털론', '김래원', '솔라', '최현석', '켄 정', '손호준', '차명석', '이준혁', '김현수', '봉준호', '트와이스다현', '이동국', '최준석', '이명박', '카이', '오재원', '지드래곤', '러블리즈지수', '송중기', '트와이스미나', '이승기', '목진화', '권태원', '박명수', '티모시 달튼', '김학범', '슬기', '유아', '리사', '박진영', '크리스 햄스워스', '강미나', '야마다 타카유키', '이범수', '존 파브르', 'Jeongmr1', '비니', '잭 니콜슨', '지코', '공효진', '류승룡', '도지한', '효정', '리암 니슨', '박항서', '박성호', '트와이스모모', '박완규', '이성민', '박희순', '지민(BTS)', '트와이스쯔위', '첸', '마동석', '권현빈', '조정치', '최무성', '스윙스', '서현진', '김광현', '류승범', '성시경', '김C', '이진호', '트와이스나연', '태연', '러블리즈예인', '최민식', '윤균상', '줄리엔강', '문별', '유재석', '길', '매드클라운', '승희', '백윤식', '철면수심', '차승원', '이말년', '김희철', '레인보우재경', '로저 무어', '서장훈', '화사', '로제', '류현진', '여자아이들수진', '전소미', '잇지예지', '박나래', '조이', '장도연', '이희준', '방시혁', '기성용', '아린', '강민경', '잇지유나', '아이린', '주진모(올드)', '유희열', '이대형', '제프리 힌턴', '트와이스정연', '찬열', '박서준', '꽈두룹', '잇섭', '정우성',

- 파일 리스트에서 '산들', '김대명', '사무엘 L 잭슨' 등 임베딩 되지 않은 리스트는 삭제되었다.

## 4. 모은 연예인 사진들과 비교하기
- 두 이미지의 임베딩 벡터 사이의 거리를 구하고 top 5 연예인들을 뽑아보자
- 두 이미지 거리 구하는 `get_distance` 함수와 거리를 비교하여 순위대로 정리해주는 `get_sort_key_func`을 이용하여 순위를 매겼다.

In [6]:
# 두 이미지의 거리구하기
import numpy as np
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

In [7]:
# name1과 name2의 거리를 비교하는 함수를 생성하되, name1은 미리 지정하고, name2는 호출시에 인자로 받도록 합니다.
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

sort_key_func = get_sort_key_func('Jeongmr1')

- `sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))`
    - 결과가 너무 길어 마크다운으로 작성하였다. 임베딩 벡터의 거리가 짧은 사진으로 내 사진 두 장이 먼저 뽑혔다.

In [9]:
def get_nearest_face(name, top=5):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x: sort_key_func(x[0]))
    
    for i in range(top+1):
        if i == 0: continue # 첫번째 이름 : 자기자신이므로 제외
        if sorted_faces[i]: print('순위 {} : 이름 ({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))

- 사진 두 장을 이용했더니 결과가 조금씩 달랐으나, top 5 내에 `조유리(아이즈원)`님, `채연(아이즈원)`이 두 번이나 들어가 있어 간단한 시각화에 쓰기로 했다.

In [10]:
get_nearest_face('Jeongmr1')

순위 1 : 이름 (Jeongmr2), 거리(0.4271515709110211)
순위 2 : 이름 (아이즈원조유리), 거리(0.4515870187184181)
순위 3 : 이름 (김고은), 거리(0.4566686216669167)
순위 4 : 이름 (조유리), 거리(0.4623773038728074)
순위 5 : 이름 (채연), 거리(0.4674529482998508)


In [11]:
get_nearest_face('Jeongmr2')

순위 1 : 이름 (우기), 거리(0.351854894971459)
순위 2 : 이름 (사쿠라), 거리(0.35752875621488356)
순위 3 : 이름 (채연), 거리(0.36521167423101414)
순위 4 : 이름 (조유리), 거리(0.3675872405928875)
순위 5 : 이름 (승희), 거리(0.37532257870615965)


## 5. 다양한 재미있는 시각화 시도해보기
- 이번 노드에서 소개했던 **PCA와 T-SNE**을 이용하여 시각화를 시도해보려 했으나, 이해가 잘 가지 않아(ㅜㅜ) **tkinter를 이용한 GUI 만들기**로 대체했다.
- GUI 새 창의 **라벨, 이미지, 타이틀을 설정**했다.
- `get_nearest_face()` 함수를 다시 풀어 값을 `DataFrame` 형식으로 담아 첫번째 이미지를 뽑아오도록 했다. `DataFrame` 형식으로 만든 이유는 5위까지 나오도록 하는 화면을 만들고자 했기 때문이다. tkinter 진행 중에 파이썬에서 처음 GUI를 다뤄봐서 모듈 등 오류가 많아 가장 간단하게 1위의 이미지민 새 창에 띄워지게 해보았다.

In [12]:
from sklearn import datasets
import pandas as pd

from sklearn.manifold import TSNE

# 거리 데이터 프레임으로 만들기
name = 'Jeongmr1'
top = 5
sort_key_func = get_sort_key_func(name)
sorted_faces = sorted(embedding_dict.items(), key=lambda x: sort_key_func(x[0]))
label = []
distance = []

for i in range(top+1):
    if i == 0: continue # 첫번째 이름 : 자기자신이므로 제외
#     if sorted_faces[i]: print('순위 {} : 이름 ({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))
    label.append(sorted_faces[i][0])
    distance.append(sort_key_func(sorted_faces[i][0]))
    
df = pd.DataFrame(
                {'name':label,
                 'distance':distance
                })
df

,name,distance
0,Jeongmr2,0.427152
1,아이즈원조유리,0.451587
2,김고은,0.456669
3,조유리,0.462377
4,채연,0.467453


In [20]:
from tkinter import *
from PIL import ImageTk

root = Tk()

root.title("내 닮은꼴 1위는?")
# root.geometry("400x500")
rank1 = df.iloc[1,0]
# font=tkinter.font.Font(family="NanumGothic", size=20, slant="italic")
label1 = Label(root, text = rank1)#, font = font)
file_n = rank1 + '.jpeg'
my_image = ImageTk.PhotoImage(file=dir_path+'/'+file_n, master=root)
lbl = Label(root, image=my_image)
label1.pack()
lbl.pack()

root.mainloop()

- 아래 그림이 그 결과로 폰트 등 옵션을 더 변경하지 못해 아쉬웠다.

![image](https://github.com/jeongmr07/aiffel/blob/master/%EB%8B%AE%EC%9D%80%EA%BC%B4.png?raw=true)

---
# 루브릭

## 1. 얼굴임베딩 벡터를 활용해 가장 닮은 연예인 Best 5 구할 수 있다.
- 닮은꼴 순위, 이름, 임베딩 거리를 포함한 Top-5 리스트가 정렬되어 출력되었다.
- `get_nearest_face()` 함수를 통해 Top-5 리스트를 정렬해 출력하였다.

## 2. 충분한 수의 이미지에 대한 시도를 통해 매우 닮은꼴의 연예인을 찾아냈다.
- 다양한 탐색을 통해 임베딩 거리 0.5 이내로 닮은 연예인을 찾아냈다.
- `Jeongmr1` 이미지 : 순위 2 : 이름 (아이즈원조유리), 거리(0.4515870187184181), `Jeongmr2` 순위 1 : 이름 (우기), 거리(0.351854894971459)로 임베딩 거리 0.5 이내의 연예인을 찾았다.

## 3. 다양하고 재미있는 결과 시각화를 시도하였다.
- matplotlib 등 다양한 시각화 도구를 하나 이상 이용해 재미있는 결과 시각화를 구현하였다.
- tkinter를 이용해서 사진을 불러와 1위 사진을 띄우는 GUI를 구현해보았다.

---
# 마무리
2021 02 04 THUR
- 임베딩 벡터의 이해를 더 깊게 하면 시각적으로 표현할 것도 많아, 더 좋은 결과가 나올 것이라는 생각이 들었다. 생각보다 얼굴 인식과정의 연구가 많이 진행되어 우리는 쉽게 사용하기만 하면 되니 배우는 사람 입장에서 응용할 거리가 많을 것 같다.
- `PCA`와 `T-SNE`의 관련 코드의 이해도가 낮아 더 이상의 시각화를 진행하지 못 한게 아쉬웠다. 그를 이용하게 되면, 텐서플로우에서 제공하는 `Projector`를 구현할 수 있을 것이다.